In [4]:
%pip install openai python-dotenv

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/appimages/cursor.AppImage -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from openai import OpenAI
import base64
import requests
import json

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

client = OpenAI(api_key=api_key)

image_path = "/home/luis/workspace/vacocam_render/test3.1.jpg"
image_path = "/home/luis/workspace/vacocam_render/output2.png"
image_path = "/home/luis/workspace/vacocam_render/source/track/presentation/prerender.mp4_4-0_4-20.png"
image_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_5-30_5-40.png"
image_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_7-40_7-50.png"
image_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_42-40_42-48.png"

metadata_path = "/home/luis/workspace/vacocam_render/source/track/presentation/prerender.mp4_4-0_4-20.txt"
metadata_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_5-30_5-40.txt"
metadata_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_7-40_7-50.txt"
metadata_path = "/home/luis/workspace/vacocam_render/experiments/vacocam/track/presentation/prerender.mp4_42-40_42-48.txt"
metadata = open(metadata_path, "r").read()

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

text = f"""
Look at this still from 10 seconds of football video. 
The colored area(s) with a letter label are clusters of ball detections across multiple frames.

The areas metadata is the following:

{metadata}

Your task is to determine which areas are ball detections belonging to the primary match, using their positioning, context, trajectory, time, etc.
There may or not be more than one match being played. Also, an object that is not a ball may have been labeled as a ball.

Pay close attention to the lines, goals and other markers to determine the bounds and action of the primary match.
The drawn arrows indicate the general direction of the detections.

You only need your visual understanding of the image as-is to accomplish this task, you dont need any additional information.

Respond with a top-level array of objects with keys "id": String, "primary": Boolean, "reason" String ["primary ball", "irrelevant ball", "not ball"].
Respond with valid JSON stripped of any display formatting (including blockquotes, etc.), comments, or explanations
Respond with labels ordered from most to least relevant to the primary match.
notalk; justgo
"""

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_image}",
            "detail": "low",
          }
        },
        {
          "type": "text",
          "text": text
        },
      ]
    }
  ],
  "max_tokens": 500,
  "temperature": 0.1,
}

print(str(text))

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


Look at this still from 10 seconds of football video. 
The colored area(s) with a letter label are clusters of ball detections across multiple frames.

The areas metadata is the following:

id, color
A, Blue
B, Pumpkin

Your task is to determine which areas are ball detections belonging to the primary match, using their positioning, context, trajectory, time, etc.
There may or not be more than one match being played. Also, an object that is not a ball may have been labeled as a ball.

Pay close attention to the lines, goals and other markers to determine the bounds and action of the primary match.
The drawn arrows indicate the general direction of the detections.

You only need your visual understanding of the image as-is to accomplish this task, you dont need any additional information.

Respond with a top-level array of objects with keys "id": String, "primary": Boolean, "reason" String ["primary ball", "irrelevant ball", "not ball"].
Respond with valid JSON stripped of any display 